# Data Science X Logistic Regression

This school project is an introduction to data cleaning, visualization and model training.

## Data loading

In [6]:
# The first thing we need to do is to download and the dataset that we will be working with.
!wget https://cdn.intra.42.fr/document/document/12379/datasets.tgz

# Then, unzip it.
!tar -xvf datasets.tgz

#And remove the zip file.
!rm datasets.tgz

--2023-09-02 16:33:55--  https://cdn.intra.42.fr/document/document/12379/datasets.tgz
Resolving cdn.intra.42.fr (cdn.intra.42.fr)... 163.172.250.39
Connecting to cdn.intra.42.fr (cdn.intra.42.fr)|163.172.250.39|:443... connected.


HTTP request sent, awaiting response... 200 OK
Length: 493568 (482K) [application/octet-stream]
Saving to: ‘datasets.tgz.1’

datasets.tgz.1      100%[===================>] 482.00K  --.-KB/s    in 0.1s    

2023-09-02 16:33:56 (3.77 MB/s) - ‘datasets.tgz.1’ saved [493568/493568]

datasets/
datasets/dataset_test.csv
datasets/dataset_train.csv


In [10]:
#Let's load the data using CSV package(since we can't use pandas).
import csv
from collections import defaultdict

columns = defaultdict(list) 

with open("./datasets/dataset_train.csv") as training_dataset_file:
    reader = csv.DictReader(training_dataset_file) # read rows into a dictionary format
    for row in reader: # read a row as {column1: value1, column2: value2,...}
        for (k,v) in row.items(): # go over each column name and value 
            columns[k].append(v) # append the value into the appropriate list based on column name k

['Left', 'Right', 'Left', 'Left', 'Left', 'Right', 'Left', 'Right', 'Left', 'Right', 'Right', 'Left', 'Right', 'Left', 'Right', 'Left', 'Left', 'Left', 'Right', 'Right', 'Left', 'Left', 'Left', 'Left', 'Left', 'Left', 'Left', 'Right', 'Left', 'Right', 'Left', 'Left', 'Right', 'Right', 'Right', 'Right', 'Right', 'Right', 'Right', 'Right', 'Left', 'Right', 'Left', 'Right', 'Left', 'Right', 'Left', 'Right', 'Left', 'Left', 'Right', 'Right', 'Right', 'Right', 'Left', 'Left', 'Left', 'Right', 'Right', 'Right', 'Right', 'Left', 'Left', 'Right', 'Right', 'Left', 'Right', 'Right', 'Left', 'Right', 'Right', 'Right', 'Right', 'Left', 'Left', 'Left', 'Left', 'Right', 'Left', 'Right', 'Left', 'Right', 'Right', 'Right', 'Right', 'Left', 'Right', 'Left', 'Left', 'Right', 'Right', 'Left', 'Left', 'Right', 'Right', 'Left', 'Left', 'Left', 'Right', 'Right', 'Left', 'Left', 'Left', 'Right', 'Right', 'Right', 'Right', 'Right', 'Left', 'Left', 'Right', 'Right', 'Right', 'Left', 'Right', 'Right', 'Right', 

## Data preprocessing

In [ ]:
# Some colums have missing values.
# Let's fill missing numerical values by the median of their respective colum.
for i in range(6, columns.items)

# Let's remove the unecessary columns date of birth, names and the index.

columns.clear